In [ ]:
%load_ext autoreload
%autoreload 2
from embedder import Embedder
import numpy as np
import pandas as pd
import tqdm
import json
import spacy
import pickle

## Load the Data

## Pruning


## Making of final data-set

## Initialize Embedder and dataset

In [ ]:
embedder = Embedder()
df = pd.read_csv('data/wikidata2.csv')
df = df.sort_values('Length')[::-1]

## Generate embeddings for dataset

In [ ]:
RUN ONLY TO CREATE NEW EMBEDDINGS

text = df.Data.values.tolist()
embeddings = []
preprocessed = embedder.preprocess(text)
tokenized = embedder.doc_tokenize(preprocessed)
batch_size = 8
for i in tqdm.trange(0,len(df),batch_size):
    embeddings.append(embedder.doc_embed(tokenized[i:i+batch_size]))

embeddings = np.concatenate(embeddings)
embeddings.shape

In [ ]:
DO NOT RUN WITHOUT CHECKING THE SAVE FILENAME.
pickle.dump(embeddings, open('data/t_embeddings.pkl', 'wb'))
np.save('data/t_embeddings.npy', embeddings)
np.savetxt('data/t_embeddings.txt', embeddings)

## Embed Queries

In [4]:
embeddings = pickle.load(open('data/t_embeddings.pkl', 'rb'))

In [5]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd


q = [
        'We advocate using the recurrent neural network transducer (RNN-T), first described in , as a more natural model for speech recognition. The RNN-T consists of two components, a transcription or encoder network and a recurrent prediction or decoder network.',
        'Papa John\'s has a new rewards program where you can earn free points for free pizza 5x faster and exclusive offers!',
        'A Convolutional Neural Network (ConvNet/CNN) is a Deep Learning algorithm which can take in an input image, assign importance (learnable weights and biases) to various aspects/objects in the image and be able to differentiate one from the other. The pre-processing required in a ConvNet is much lower as compared to other classification algorithms. While in primitive methods filters are hand-engineered, with enough training, ConvNets have the ability to learn these filters/characteristics.'
    ]

qp = embedder.preprocess(q)
qt = embedder.doc_tokenize(q)
qe = embedder.doc_embed(qt)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 62.67it/s]
W0301 23:04:12.295218 11940 deprecation_wrapper.py:119] From C:\Users\spark\PycharmProjects\capstone\embedder\embedder.py:30: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0301 23:04:13.721509 11940 deprecation_wrapper.py:119] From C:\Users\spark\PycharmProjects\capstone\embedder\embedder.py:31: The name tf.global_variables_initializer is deprecated. Please use tf.compat.v1.global_variables_initializer instead.

W0301 23:04:14.330683 11940 deprecation_wrapper.py:119] From C:\Users\spark\PycharmProjects\capstone\embedder\embedder.py:32: The name tf.tables_initializer is deprecated. Please use tf.compat.v1.tables_initializer instead.



In [6]:
if len(embeddings.shape) != 2:
    embeddings = embeddings.reshape(-1,1024)

In [7]:
concepts = df.Concept.values
for ix in range(len(qe)):
    cosine_similarities = pd.Series(cosine_similarity(qe[ix].reshape(1,-1), embeddings).flatten())
    output =""
    for i,j in cosine_similarities.nlargest(25).iteritems():
        print(str(i), str(j), concepts[i])

    print(output)

307 0.8562078218616183 https://en.wikipedia.org/wiki/Long_short-term_memory
394 0.8440139867615654 https://en.wikipedia.org/wiki/Neural_network
757 0.8409550977824336 https://en.wikipedia.org/wiki/Feedforward_neural_network
398 0.8402388880231688 https://en.wikipedia.org/wiki/Rectifier_(neural_networks)
129 0.8401017098735793 https://en.wikipedia.org/wiki/Convolutional_neural_network
522 0.8359023014867027 https://en.wikipedia.org/wiki/ADALINE
383 0.8349015862436697 https://en.wikipedia.org/wiki/Bidirectional_recurrent_neural_networks
232 0.8336328560599863 https://en.wikipedia.org/wiki/Recurrent_neural_network
495 0.8284063522383736 https://en.wikipedia.org/wiki/Multilayer_perceptron
237 0.8264054034019389 https://en.wikipedia.org/wiki/Neural_Turing_machine
552 0.8204692283856505 https://en.wikipedia.org/wiki/Non-negative_matrix_factorization
324 0.8194919599246133 https://en.wikipedia.org/wiki/Capsule_neural_network
480 0.8171801757380083 https://en.wikipedia.org/wiki/Multiple_discri